In [ ]:
import re
from siphon.catalog import TDSCatalog
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import dask.array as da
from cartopy import crs as ccrs, feature as cfeature
from scipy.stats import skew, kurtosis

# Functions

In [ ]:
def compute_dask_on_gpu(dataset):
    """
    将dask array计算并转移到GPU，然后重建为完整的xarray Dataset
    """
    # 1. 计算dask array
    computed_dataset = dataset.compute()
    
    # 2. 转到GPU
    gpu_data = {
        var: cp.asarray(computed_dataset[var].values) 
        for var in ['tas', 'pr']
    }
    
    # 3. 转回CPU并重建Dataset
    cpu_data = {
        var: (('time', 'lat', 'lon'), cp.asnumpy(arr))
        for var, arr in gpu_data.items()
    }
    
    # 4. 创建新的Dataset
    result_dataset = xr.Dataset(
        data_vars=cpu_data,
        coords=computed_dataset.coords,
        attrs=computed_dataset.attrs
    )
    
    return result_dataset

In [ ]:
def calculate_wet_bulb_temperature(temp_kelvin, relative_humidity):
    temp_celsius = temp_kelvin - 273.15
    e_s = 6.105 * np.exp(17.27 * temp_celsius / (237.7 + temp_celsius))
    e = (relative_humidity / 100.0) * e_s
    wet_bulb_celsius = temp_celsius * np.arctan(0.151977 * ((relative_humidity + 8.313659) ** 0.5)) + \
                       np.arctan(temp_celsius + relative_humidity) - \
                       np.arctan(relative_humidity - 1.676331) + \
                       0.00391838 * (relative_humidity ** 1.5) * np.arctan(0.023101 * relative_humidity) - 4.686035

    return wet_bulb_celsius

# ACCESS-CM2

In [ ]:
base_url_template = "https://ds.nccs.nasa.gov/thredds/catalog/AMES/NEX/GDDP-CMIP6/ACCESS-CM2/{scenario}/r1i1p1f1/{variable}/catalog.xml"
scenarios = {
    'historical': (1981, 2010),
    'ssp245': (2071, 2100),
    'ssp585': (2071, 2100)
}
variables = ['tas', 'tasmax', 'tasmin', 'pr', 'hurs', 'sfcWind']
year_pattern = re.compile(r'_gn_(\d{4})\.nc$')

datasets = {}

for scenario, (start_year, end_year) in scenarios.items():
    all_data_urls = []

    for var in variables:
        catalog_url = base_url_template.format(scenario=scenario, variable=var)
        catalog = TDSCatalog(catalog_url)
        
        for dataset_name in catalog.datasets:
            match = year_pattern.search(dataset_name)
            if match:
                year = int(match.group(1))
                if start_year <= year <= end_year:
                    data_urls = catalog.datasets[dataset_name].access_urls['OPENDAP']
                    all_data_urls.append(data_urls)
    
    datasets[scenario] = xr.open_mfdataset(all_data_urls, combine='by_coords')

Accesscm2_historical = datasets['historical']
Accesscm2_ssp245 = datasets['ssp245']
Accesscm2_ssp585 = datasets['ssp585']

In [ ]:
us_lat_range = slice(24, 50)  # 纬度范围
us_lon_range = slice(360 - 125, 360 - 66)  # 经度从 360 度表示的 -125°W 至 -66°W

us_hist_2 = Accesscm2_historical[['tas', 'pr']].sel(lat=us_lat_range, lon=us_lon_range)
us_245_2 = Accesscm2_ssp245[['tas', 'pr']].sel(lat=us_lat_range, lon=us_lon_range)
us_585_2 = Accesscm2_ssp585[['tas', 'pr']].sel(lat=us_lat_range, lon=us_lon_range)

us_hist_computed = compute_dask_on_gpu(us_hist_2)
us_hist_copy = us_hist_computed.copy()
us_hist_copy.attrs = {}
us_hist_copy.to_netcdf('A_us_hist_2.nc')

us_245_computed = compute_dask_on_gpu(us_245_2)
us_245_copy = us_245_computed.copy()
us_245_copy.attrs = {}
us_245_copy.to_netcdf('A_us_245_2.nc')

us_585_computed = compute_dask_on_gpu(us_585_2)
us_585_copy = us_585_computed.copy()
us_585_copy.attrs = {}
us_585_copy.to_netcdf('A_us_585_2.nc')

In [ ]:
us_hist = xr.open_dataset("A_us_hist.nc")
us_245 = xr.open_dataset("A_us_245.nc")
us_585 = xr.open_dataset("A_us_585.nc")

wetbulb_hist = xr.apply_ufunc(calculate_wet_bulb_temperature, us_hist['tasmax'], us_hist['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, us_245['tasmax'], us_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, us_585['tasmax'], us_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

hurs_control = us_hist['hurs'].copy()
hurs_control['time'] = us_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, us_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, us_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

us_hist['wetbulb'] = wetbulb_hist
us_245['wetbulb'] = wetbulb_245
us_585['wetbulb'] = wetbulb_585
us_245['wetbulb_control'] = wetbulb_245_control
us_585['wetbulb_control'] = wetbulb_585_control

us_hist_copy = us_hist.copy()
us_hist_copy.attrs = {}
us_hist_copy.to_netcdf('A_us_hist.nc')

us_245_copy = us_245.copy()
us_245_copy.attrs = {}
us_245_copy.to_netcdf('A_us_245.nc')

us_585_copy = us_585.copy()
us_585_copy.attrs = {}
us_585_copy.to_netcdf('A_us_585.nc')

## Miami

In [ ]:
Miami_historical = Accesscm2_historical.sel(lat=25.7617, lon=360 - 80.1918, method="nearest").chunk({'time': -1})
Miami_245 = Accesscm2_ssp245.sel(lat=25.7617, lon=360 - 80.1918, method="nearest").chunk({'time': -1})
Miami_585 = Accesscm2_ssp585.sel(lat=25.7617, lon=360 - 80.1918, method="nearest").chunk({'time': -1})

Miami_historical['range'] = Miami_historical.tasmax - Miami_historical.tasmin
Miami_245['range'] = Miami_245.tasmax - Miami_245.tasmin
Miami_585['range'] = Miami_585.tasmax - Miami_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_historical['tasmax'], Miami_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_245['tasmax'], Miami_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_585['tasmax'], Miami_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

#对照
hurs_control = Miami_historical['hurs'].copy()
hurs_control['time'] = Miami_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

Miami_historical['wetbulb'] = wetbulb_historical
Miami_245['wetbulb'] = wetbulb_245
Miami_585['wetbulb'] = wetbulb_585
Miami_245['wetbulb_control'] = wetbulb_245_control
Miami_585['wetbulb_control'] = wetbulb_585_control

Miami_historical = Miami_historical.compute()
Miami_245 = Miami_245.compute()
Miami_585 = Miami_585.compute()

MI_historical_copy = Miami_historical.copy()
MI_historical_copy.attrs = {}
MI_historical_copy.to_netcdf('A_MI_hist.nc')

MI_245_copy = Miami_245.copy()
MI_245_copy.attrs = {}
MI_245_copy.to_netcdf('A_Miami_245.nc')

MI_585_copy = Miami_585.copy()
MI_585_copy.attrs = {}
MI_585_copy.to_netcdf('A_Miami_585.nc')

## Phoenix

In [ ]:
# 33.4484° N, 112.0740° W
PX_historical = Accesscm2_historical.sel(lat=33.4484, lon=360 - 112.0740, method="nearest").chunk({'time': -1})
PX_245 = Accesscm2_ssp245.sel(lat=33.4484, lon=360 - 112.0740, method="nearest").chunk({'time': -1})
PX_585 = Accesscm2_ssp585.sel(lat=33.4484, lon=360 - 112.0740, method="nearest").chunk({'time': -1})

PX_historical['range'] = PX_historical.tasmax - PX_historical.tasmin
PX_245['range'] = PX_245.tasmax - PX_245.tasmin
PX_585['range'] = PX_585.tasmax - PX_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_historical['tasmax'], PX_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_245['tasmax'], PX_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_585['tasmax'], PX_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

hurs_control = PX_historical['hurs'].copy()
hurs_control['time'] = PX_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

PX_historical['wetbulb'] = wetbulb_historical
PX_245['wetbulb'] = wetbulb_245
PX_585['wetbulb'] = wetbulb_585
PX_245['wetbulb_control'] = wetbulb_245_control
PX_585['wetbulb_control'] = wetbulb_585_control

PX_historical = PX_historical.compute()
PX_245 = PX_245.compute()
PX_585 = PX_585.compute()

PX_historical_copy = PX_historical.copy()
PX_historical_copy.attrs = {}
PX_historical_copy.to_netcdf('A_PX_historical.nc')

PX_245_copy = PX_245.copy()
PX_245_copy.attrs = {}
PX_245_copy.to_netcdf('A_PX_245.nc')

PX_585_copy = PX_585.copy()
PX_585_copy.attrs = {}
PX_585_copy.to_netcdf('A_PX_585.nc')

## Los Angeles

In [ ]:
LA_historical = Accesscm2_historical.sel(lat=34.0549, lon=360 - 118.2426, method="nearest").chunk({'time': -1})
LA_245 = Accesscm2_ssp245.sel(lat=34.0549, lon=360 - 118.2426, method="nearest").chunk({'time': -1})
LA_585 = Accesscm2_ssp585.sel(lat=34.0549, lon=360 - 118.2426, method="nearest").chunk({'time': -1})

LA_historical['range'] = LA_historical.tasmax - LA_historical.tasmin
LA_245['range'] = LA_245.tasmax - LA_245.tasmin
LA_585['range'] = LA_585.tasmax - LA_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_historical['tasmax'], LA_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_245['tasmax'], LA_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_585['tasmax'], LA_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

hurs_control = LA_historical['hurs'].copy()
hurs_control['time'] = LA_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

LA_historical['wetbulb'] = wetbulb_historical
LA_245['wetbulb'] = wetbulb_245
LA_585['wetbulb'] = wetbulb_585
LA_245['wetbulb_control'] = wetbulb_245_control
LA_585['wetbulb_control'] = wetbulb_585_control

LA_historical = LA_historical.compute()
LA_245 = LA_245.compute()
LA_585 = LA_585.compute()

LA_historical_copy = LA_historical.copy()
LA_historical_copy.attrs = {}
LA_historical_copy.to_netcdf('A_LA_historical.nc')

LA_245_copy = LA_245.copy()
LA_245_copy.attrs = {}
LA_245_copy.to_netcdf('A_LA_245.nc')

LA_585_copy = LA_585.copy()
LA_585_copy.attrs = {}
LA_585_copy.to_netcdf('A_LA_585.nc')

## Chicago

In [ ]:
# 41.8781° N, 87.6298° W
CG_historical = Accesscm2_historical.sel(lat=41.8781, lon=360 - 87.6298, method="nearest").chunk({'time': -1})
CG_245 = Accesscm2_ssp245.sel(lat=41.8781, lon=360 - 87.6298, method="nearest").chunk({'time': -1})
CG_585 = Accesscm2_ssp585.sel(lat=41.8781, lon=360 - 87.6298, method="nearest").chunk({'time': -1})

CG_historical['range'] = CG_historical.tasmax - CG_historical.tasmin
CG_245['range'] = CG_245.tasmax - CG_245.tasmin
CG_585['range'] = CG_585.tasmax - CG_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_historical['tasmax'], CG_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_245['tasmax'], CG_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_585['tasmax'], CG_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

hurs_control = CG_historical['hurs'].copy()
hurs_control['time'] = CG_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

CG_historical['wetbulb'] = wetbulb_historical
CG_245['wetbulb'] = wetbulb_245
CG_585['wetbulb'] = wetbulb_585
CG_245['wetbulb_control'] = wetbulb_245_control
CG_585['wetbulb_control'] = wetbulb_585_control

CG_historical = CG_historical.compute()
CG_245 = CG_245.compute()
CG_585 = CG_585.compute()

CG_historical_copy = CG_historical.copy()
CG_historical_copy.attrs = {}
CG_historical_copy.to_netcdf('A_CG_historical.nc')

CG_245_copy = CG_245.copy()
CG_245_copy.attrs = {}
CG_245_copy.to_netcdf('A_CG_245.nc')

CG_585_copy = CG_585.copy()
CG_585_copy.attrs = {}
CG_585_copy.to_netcdf('A_CG_585.nc')

# GFDL-CM4

In [ ]:
base_url_template = "https://ds.nccs.nasa.gov/thredds/catalog/AMES/NEX/GDDP-CMIP6/GFDL-CM4/{scenario}/r1i1p1f1/{variable}/catalog.xml"
scenarios = {
    'historical': (1981, 2010),
    'ssp245': (2071, 2100),
    'ssp585': (2071, 2100)
}
variables = ['tas', 'tasmax', 'tasmin', 'pr', 'hurs', 'sfcWind']
year_pattern = re.compile(r'_gr1_(\d{4})\.nc$')

datasets = {}

for scenario, (start_year, end_year) in scenarios.items():
    all_data_urls = []

    for var in variables:
        catalog_url = base_url_template.format(scenario=scenario, variable=var)
        catalog = TDSCatalog(catalog_url)
        
        for dataset_name in catalog.datasets:
            match = year_pattern.search(dataset_name)
            if match:
                year = int(match.group(1))
                if start_year <= year <= end_year:
                    data_urls = catalog.datasets[dataset_name].access_urls['OPENDAP']
                    all_data_urls.append(data_urls)
    
    datasets[scenario] = xr.open_mfdataset(all_data_urls, combine='by_coords')

GFDL_historical = datasets['historical']
GFDL_245 = datasets['ssp245']
GFDL_585 = datasets['ssp585']

## US

In [ ]:
us_lat_range = slice(24, 50)  
us_lon_range = slice(360 - 125, 360 - 66) 

us_hist = GFDL_historical[['tas', 'pr']].sel(lat=us_lat_range, lon=us_lon_range)
us_245 = GFDL_245[['tas', 'pr']].sel(lat=us_lat_range, lon=us_lon_range)
us_585 = GFDL_585[['tas', 'pr']].sel(lat=us_lat_range, lon=us_lon_range)

us_hist_computed = compute_dask_on_gpu(us_hist)
us_hist_copy = us_hist_computed.copy()
us_hist_copy.attrs = {}
us_hist_copy.to_netcdf('G_us_hist_2.nc')

us_245_computed = compute_dask_on_gpu(us_245)
us_245_copy = us_245_computed.copy()
us_245_copy.attrs = {}
us_245_copy.to_netcdf('G_us_245_2.nc')

us_585_computed = compute_dask_on_gpu(us_585)
us_585_copy = us_585_computed.copy()
us_585_copy.attrs = {}
us_585_copy.to_netcdf('G_us_585_2.nc')

In [ ]:
us_hist = xr.open_dataset("G_us_hist.nc")
us_245 = xr.open_dataset("G_us_245.nc")
us_585 = xr.open_dataset("G_us_585.nc")

wetbulb_hist = xr.apply_ufunc(calculate_wet_bulb_temperature, us_hist['tasmax'], us_hist['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, us_245['tasmax'], us_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, us_585['tasmax'], us_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

hurs_control = us_hist['hurs'].copy()
hurs_control['time'] = us_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, us_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, us_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

us_hist['wetbulb'] = wetbulb_hist
us_245['wetbulb'] = wetbulb_245
us_585['wetbulb'] = wetbulb_585
us_245['wetbulb_control'] = wetbulb_245_control
us_585['wetbulb_control'] = wetbulb_585_control

us_hist_copy = us_hist.copy()
us_hist_copy.attrs = {}
us_hist_copy.to_netcdf('G_us_hist.nc')

us_245_copy = us_245.copy()
us_245_copy.attrs = {}
us_245_copy.to_netcdf('G_us_245.nc')

us_585_copy = us_585.copy()
us_585_copy.attrs = {}
us_585_copy.to_netcdf('G_us_585.nc')

## Miami

In [ ]:
Miami_historical = GFDL_historical.sel(lat=25.7617, lon=360 - 80.1918, method="nearest").chunk({'time': -1})
Miami_245 = GFDL_245.sel(lat=25.7617, lon=360 - 80.1918, method="nearest").chunk({'time': -1})
Miami_585 = GFDL_585.sel(lat=25.7617, lon=360 - 80.1918, method="nearest").chunk({'time': -1})

Miami_historical['range'] = Miami_historical.tasmax - Miami_historical.tasmin
Miami_245['range'] = Miami_245.tasmax - Miami_245.tasmin
Miami_585['range'] = Miami_585.tasmax - Miami_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_historical['tasmax'], Miami_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_245['tasmax'], Miami_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_585['tasmax'], Miami_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

#对照
hurs_control = Miami_historical['hurs'].copy()
hurs_control['time'] = Miami_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

Miami_historical['wetbulb'] = wetbulb_historical
Miami_245['wetbulb'] = wetbulb_245
Miami_585['wetbulb'] = wetbulb_585
Miami_245['wetbulb_control'] = wetbulb_245_control
Miami_585['wetbulb_control'] = wetbulb_585_control

Miami_historical = Miami_historical.compute()
Miami_245 = Miami_245.compute()
Miami_585 = Miami_585.compute()

MI_historical_copy = Miami_historical.copy()
MI_historical_copy.attrs = {}
MI_historical_copy.to_netcdf('G_MI_hist.nc')

MI_245_copy = Miami_245.copy()
MI_245_copy.attrs = {}
MI_245_copy.to_netcdf('G_Miami_245.nc')

MI_585_copy = Miami_585.copy()
MI_585_copy.attrs = {}
MI_585_copy.to_netcdf('G_Miami_585.nc')

## Phoenix

In [ ]:
# 33.4484° N, 112.0740° W
PX_historical = GFDL_historical.sel(lat=33.4484, lon=360 - 112.0740, method="nearest").chunk({'time': -1})
PX_245 = GFDL_245.sel(lat=33.4484, lon=360 - 112.0740, method="nearest").chunk({'time': -1})
PX_585 = GFDL_585.sel(lat=33.4484, lon=360 - 112.0740, method="nearest").chunk({'time': -1})

PX_historical['range'] = PX_historical.tasmax - PX_historical.tasmin
PX_245['range'] = PX_245.tasmax - PX_245.tasmin
PX_585['range'] = PX_585.tasmax - PX_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_historical['tasmax'], PX_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_245['tasmax'], PX_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_585['tasmax'], PX_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

hurs_control = PX_historical['hurs'].copy()
hurs_control['time'] = PX_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

PX_historical['wetbulb'] = wetbulb_historical
PX_245['wetbulb'] = wetbulb_245
PX_585['wetbulb'] = wetbulb_585
PX_245['wetbulb_control'] = wetbulb_245_control
PX_585['wetbulb_control'] = wetbulb_585_control

PX_historical = PX_historical.compute()
PX_historical_copy = PX_historical.copy()
PX_historical_copy.attrs = {}
PX_historical_copy.to_netcdf('G_PX_hist.nc')

PX_245 = PX_245.compute()
PX_245_copy = PX_245.copy()
PX_245_copy.attrs = {}
PX_245_copy.to_netcdf('G_PX_245.nc')

PX_585 = PX_585.compute()
PX_585_copy = PX_585.copy()
PX_585_copy.attrs = {}
PX_585_copy.to_netcdf('G_PX_585.nc')

## Los Angeles

In [ ]:
LA_historical = GFDL_historical.sel(lat=34.0549, lon=360 - 118.2426, method="nearest").chunk({'time': -1})
LA_245 = GFDL_245.sel(lat=34.0549, lon=360 - 118.2426, method="nearest").chunk({'time': -1})
LA_585 = GFDL_585.sel(lat=34.0549, lon=360 - 118.2426, method="nearest").chunk({'time': -1})

LA_historical['range'] = LA_historical.tasmax - LA_historical.tasmin
LA_245['range'] = LA_245.tasmax - LA_245.tasmin
LA_585['range'] = LA_585.tasmax - LA_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_historical['tasmax'], LA_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_245['tasmax'], LA_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_585['tasmax'], LA_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

#对照
hurs_control = LA_historical['hurs'].copy()
hurs_control['time'] = LA_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

LA_historical['wetbulb'] = wetbulb_historical
LA_245['wetbulb'] = wetbulb_245
LA_585['wetbulb'] = wetbulb_585
LA_245['wetbulb_control'] = wetbulb_245_control
LA_585['wetbulb_control'] = wetbulb_585_control

LA_historical = LA_historical.compute()
LA_historical_copy = LA_historical.copy()
LA_historical_copy.attrs = {}
LA_historical_copy.to_netcdf('G_LA_hist.nc')

LA_245 = LA_245.compute()
LA_245_copy = LA_245.copy()
LA_245_copy.attrs = {}
LA_245_copy.to_netcdf('G_LA_245.nc')

LA_585 = LA_585.compute()
LA_585_copy = LA_585.copy()
LA_585_copy.attrs = {}
LA_585_copy.to_netcdf('G_LA_585.nc')

## Chicago

In [ ]:
# 41.8781° N, 87.6298° W
CG_historical = GFDL_historical.sel(lat=41.8781, lon=360 - 87.6298, method="nearest").chunk({'time': -1})
CG_245 = GFDL_245.sel(lat=41.8781, lon=360 - 87.6298, method="nearest").chunk({'time': -1})
CG_585 = GFDL_585.sel(lat=41.8781, lon=360 - 87.6298, method="nearest").chunk({'time': -1})

CG_historical['range'] = CG_historical.tasmax - CG_historical.tasmin
CG_245['range'] = CG_245.tasmax - CG_245.tasmin
CG_585['range'] = CG_585.tasmax - CG_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_historical['tasmax'], CG_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_245['tasmax'], CG_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_585['tasmax'], CG_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

#对照
hurs_control = CG_historical['hurs'].copy()
hurs_control['time'] = CG_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

CG_historical['wetbulb'] = wetbulb_historical
CG_245['wetbulb'] = wetbulb_245
CG_585['wetbulb'] = wetbulb_585
CG_245['wetbulb_control'] = wetbulb_245_control
CG_585['wetbulb_control'] = wetbulb_585_control

CG_historical = CG_historical.compute()
CG_historical_copy = CG_historical.copy()
CG_historical_copy.attrs = {}
CG_historical_copy.to_netcdf('G_CG_hist.nc')

CG_245 = CG_245.compute()
CG_245_copy = CG_245.copy()
CG_245_copy.attrs = {}
CG_245_copy.to_netcdf('G_CG_245.nc')

CG_585 = CG_585.compute()
CG_585_copy = CG_585.copy()
CG_585_copy.attrs = {}
CG_585_copy.to_netcdf('G_CG_585.nc')

# UKESM1-0-LL

In [ ]:
base_url_template = "https://ds.nccs.nasa.gov/thredds/catalog/AMES/NEX/GDDP-CMIP6/UKESM1-0-LL/{scenario}/r1i1p1f2/{variable}/catalog.xml"
scenarios = {
    'historical': (1981, 2010),
    'ssp245': (2071, 2100),
    'ssp585': (2071, 2100)
}
variables = ['tas', 'tasmax', 'tasmin', 'pr', 'hurs', 'sfcWind']
year_pattern = re.compile(r'_gn_(\d{4})\.nc$')

datasets = {}

for scenario, (start_year, end_year) in scenarios.items():
    all_data_urls = []

    for var in variables:
        catalog_url = base_url_template.format(scenario=scenario, variable=var)
        catalog = TDSCatalog(catalog_url)
        
        for dataset_name in catalog.datasets:
            match = year_pattern.search(dataset_name)
            if match:
                year = int(match.group(1))
                if start_year <= year <= end_year:
                    data_urls = catalog.datasets[dataset_name].access_urls['OPENDAP']
                    all_data_urls.append(data_urls)
    
    datasets[scenario] = xr.open_mfdataset(all_data_urls, combine='by_coords')

UKESM_historical = datasets['historical']
UKESM_245 = datasets['ssp245']
UKESM_585 = datasets['ssp585']

## US

In [ ]:
us_lat_range = slice(24, 50)  
us_lon_range = slice(360 - 125, 360 - 66) 

us_hist = UKESM_historical[['tas', 'pr']].sel(lat=us_lat_range, lon=us_lon_range)
us_245 = UKESM_245[['tas', 'pr']].sel(lat=us_lat_range, lon=us_lon_range)
us_585 = UKESM_585[['tas', 'pr']].sel(lat=us_lat_range, lon=us_lon_range)

us_hist_computed = compute_dask_on_gpu(us_hist)
us_hist_copy = us_hist_computed.copy()
us_hist_copy.attrs = {}
us_hist_copy.to_netcdf('U_us_hist_2.nc')

us_245_computed = compute_dask_on_gpu(us_245)
us_245_copy = us_245_computed.copy()
us_245_copy.attrs = {}
us_245_copy.to_netcdf('U_us_245_2.nc')

us_585_computed = compute_dask_on_gpu(us_585)
us_585_copy = us_585_computed.copy()
us_585_copy.attrs = {}
us_585_copy.to_netcdf('U_us_585_2.nc')

In [ ]:
us_hist = xr.open_dataset("U_us_hist.nc")
us_245 = xr.open_dataset("U_us_245.nc")
us_585 = xr.open_dataset("U_us_585.nc")

wetbulb_hist = xr.apply_ufunc(calculate_wet_bulb_temperature, us_hist['tasmax'], us_hist['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, us_245['tasmax'], us_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, us_585['tasmax'], us_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

hurs_control = us_hist['hurs'].copy()
hurs_control['time'] = us_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, us_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, us_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

us_hist['wetbulb'] = wetbulb_hist
us_245['wetbulb'] = wetbulb_245
us_585['wetbulb'] = wetbulb_585
us_245['wetbulb_control'] = wetbulb_245_control
us_585['wetbulb_control'] = wetbulb_585_control

us_hist_copy = us_hist.copy()
us_hist_copy.attrs = {}
us_hist_copy.to_netcdf('G_us_hist_1.nc')

us_245_copy = us_245.copy()
us_245_copy.attrs = {}
us_245_copy.to_netcdf('G_us_245_1.nc')

us_585_copy = us_585.copy()
us_585_copy.attrs = {}
us_585_copy.to_netcdf('G_us_585_1.nc')

## Miami

In [ ]:
Miami_historical = UKESM_historical.sel(lat=25.7617, lon=360 - 80.1918, method="nearest").chunk({'time': -1})
Miami_245 = UKESM_245.sel(lat=25.7617, lon=360 - 80.1918, method="nearest").chunk({'time': -1})
Miami_585 = UKESM_585.sel(lat=25.7617, lon=360 - 80.1918, method="nearest").chunk({'time': -1})

Miami_historical['range'] = Miami_historical.tasmax - Miami_historical.tasmin
Miami_245['range'] = Miami_245.tasmax - Miami_245.tasmin
Miami_585['range'] = Miami_585.tasmax - Miami_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_historical['tasmax'], Miami_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_245['tasmax'], Miami_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_585['tasmax'], Miami_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

#对照
hurs_control = Miami_historical['hurs'].copy()
hurs_control['time'] = Miami_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, Miami_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

Miami_historical['wetbulb'] = wetbulb_historical
Miami_245['wetbulb'] = wetbulb_245
Miami_585['wetbulb'] = wetbulb_585
Miami_245['wetbulb_control'] = wetbulb_245_control
Miami_585['wetbulb_control'] = wetbulb_585_control

Miami_historical = Miami_historical.compute()
MI_historical_copy = Miami_historical.copy()
MI_historical_copy.attrs = {}
MI_historical_copy.to_netcdf('U_MI_hist.nc')

Miami_245 = Miami_245.compute()
MI_245_copy = Miami_245.copy()
MI_245_copy.attrs = {}
MI_245_copy.to_netcdf('U_Miami_245.nc')

Miami_585 = Miami_585.compute()
MI_585_copy = Miami_585.copy()
MI_585_copy.attrs = {}
MI_585_copy.to_netcdf('U_Miami_585.nc')

## Phoenix

In [ ]:
# 33.4484° N, 112.0740° W
PX_historical = UKESM_historical.sel(lat=33.4484, lon=360 - 112.0740, method="nearest").chunk({'time': -1})
PX_245 = UKESM_245.sel(lat=33.4484, lon=360 - 112.0740, method="nearest").chunk({'time': -1})
PX_585 = UKESM_585.sel(lat=33.4484, lon=360 - 112.0740, method="nearest").chunk({'time': -1})

PX_historical['range'] = PX_historical.tasmax - PX_historical.tasmin
PX_245['range'] = PX_245.tasmax - PX_245.tasmin
PX_585['range'] = PX_585.tasmax - PX_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_historical['tasmax'], PX_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_245['tasmax'], PX_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_585['tasmax'], PX_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

hurs_control = PX_historical['hurs'].copy()
hurs_control['time'] = PX_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, PX_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

PX_historical['wetbulb'] = wetbulb_historical
PX_245['wetbulb'] = wetbulb_245
PX_585['wetbulb'] = wetbulb_585
PX_245['wetbulb_control'] = wetbulb_245_control
PX_585['wetbulb_control'] = wetbulb_585_control

PX_historical = PX_historical.compute()
PX_historical_copy = PX_historical.copy()
PX_historical_copy.attrs = {}
PX_historical_copy.to_netcdf('U_PX_hist.nc')

PX_245 = PX_245.compute()
PX_245_copy = PX_245.copy()
PX_245_copy.attrs = {}
PX_245_copy.to_netcdf('U_PX_245.nc')

PX_585 = PX_585.compute()
PX_585_copy = PX_585.copy()
PX_585_copy.attrs = {}
PX_585_copy.to_netcdf('U_PX_585.nc')

## Los Angeles

In [ ]:
LA_historical = UKESM_historical.sel(lat=34.0549, lon=360 - 118.2426, method="nearest").chunk({'time': -1})
LA_245 = UKESM_245.sel(lat=34.0549, lon=360 - 118.2426, method="nearest").chunk({'time': -1})
LA_585 = UKESM_585.sel(lat=34.0549, lon=360 - 118.2426, method="nearest").chunk({'time': -1})

LA_historical['range'] = LA_historical.tasmax - LA_historical.tasmin
LA_245['range'] = LA_245.tasmax - LA_245.tasmin
LA_585['range'] = LA_585.tasmax - LA_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_historical['tasmax'], LA_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_245['tasmax'], LA_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_585['tasmax'], LA_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

#对照
hurs_control = LA_historical['hurs'].copy()
hurs_control['time'] = LA_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, LA_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

LA_historical['wetbulb'] = wetbulb_historical
LA_245['wetbulb'] = wetbulb_245
LA_585['wetbulb'] = wetbulb_585
LA_245['wetbulb_control'] = wetbulb_245_control
LA_585['wetbulb_control'] = wetbulb_585_control

LA_historical = LA_historical.compute()
LA_historical_copy = LA_historical.copy()
LA_historical_copy.attrs = {}
LA_historical_copy.to_netcdf('U_LA_hist.nc')

LA_245 = LA_245.compute()
LA_245_copy = LA_245.copy()
LA_245_copy.attrs = {}
LA_245_copy.to_netcdf('U_LA_245.nc')

LA_585 = LA_585.compute()
LA_585_copy = LA_585.copy()
LA_585_copy.attrs = {}
LA_585_copy.to_netcdf('U_LA_585.nc')

## Chicago

In [ ]:
# 41.8781° N, 87.6298° W
CG_historical = UKESM_historical.sel(lat=41.8781, lon=360 - 87.6298, method="nearest").chunk({'time': -1})
CG_245 = UKESM_245.sel(lat=41.8781, lon=360 - 87.6298, method="nearest").chunk({'time': -1})
CG_585 = UKESM_585.sel(lat=41.8781, lon=360 - 87.6298, method="nearest").chunk({'time': -1})

CG_historical['range'] = CG_historical.tasmax - CG_historical.tasmin
CG_245['range'] = CG_245.tasmax - CG_245.tasmin
CG_585['range'] = CG_585.tasmax - CG_585.tasmin

wetbulb_historical = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_historical['tasmax'], CG_historical['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_245 = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_245['tasmax'], CG_245['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585 = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_585['tasmax'], CG_585['hurs'], 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

#对照
hurs_control = CG_historical['hurs'].copy()
hurs_control['time'] = CG_245['tasmax']['time']

wetbulb_245_control = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_245['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])
wetbulb_585_control = xr.apply_ufunc(calculate_wet_bulb_temperature, CG_585['tasmax'], hurs_control, 
    input_core_dims=[[], []],  vectorize=True, dask="parallelized", output_dtypes=[float])

CG_historical['wetbulb'] = wetbulb_historical
CG_245['wetbulb'] = wetbulb_245
CG_585['wetbulb'] = wetbulb_585
CG_245['wetbulb_control'] = wetbulb_245_control
CG_585['wetbulb_control'] = wetbulb_585_control

CG_historical = CG_historical.compute()
CG_historical_copy = CG_historical.copy()
CG_historical_copy.attrs = {}
CG_historical_copy.to_netcdf('U_CG_hist.nc')

CG_245 = CG_245.compute()
CG_245_copy = CG_245.copy()
CG_245_copy.attrs = {}
CG_245_copy.to_netcdf('U_CG_245.nc')

CG_585 = CG_585.compute()
CG_585_copy = CG_585.copy()
CG_585_copy.attrs = {}
CG_585_copy.to_netcdf('U_CG_585.nc')